# Anish Moorthy CS4641 Project 2 Code

Heavily based on the example code from the mlrose package.

Python > Java :P

### Setup Code

Depending on which combination of problem and algorithms you want to test, run the corresponding set of cells. There should be one set of cells for choosing the problem type. 

mlrose is a Python package for applying some of the most common randomized optimization and search algorithms to a range of different optimization problems, over both discrete- and continuous-valued parameter spaces. This notebook contains the examples used in the mlrose tutorial.

In [0]:
#############################
# Library Installation Cell #
#############################

!pip uninstall -y mlrose
!pip install git+https://github.com/pipsqueaker/mlrose.git@master

Skipping mlrose as it is not installed.
  Cloning https://github.com/pipsqueaker/mlrose.git (to revision master) to /tmp/pip-req-build-h113mg4w
  Stored in directory: /tmp/pip-ephem-wheel-cache-j6wu0mwq/wheels/66/23/97/1ea12771a9a08daf8b18e55d0415983896b8b6f1d3a5e38bbb
Successfully built mlrose


In [0]:
import mlrose
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import time
import matplotlib.pyplot as plt
import pylab
import copy
import keras
%matplotlib inline

P_FOURPEAKS = "fourpeaks"
P_QUEENS = "queens"
P_FLIPFLOP = "flipflop"

rhc_specs = []
ga_specs = []
sa_specs = []
mimic_specs = []

def jagged_average(history_list):
    max_history_length = max([h.size for h in history_list])
    extended_histories = np.array([np.append(h, [np.nan] * (max_history_length - h.size)) 
                          for h in history_list])
    
    history_mask = extended_histories.copy()
    history_mask[~np.isnan(history_mask)] = 1
    extended_histories[np.isnan(extended_histories)] = 0
    
    return np.sum(extended_histories, axis=0) / np.sum(history_mask, axis=0)

### Experiment Selection (choose one cell to run)

In [0]:
# Four Peaks

PROBLEM_SIZE = 8

MAXIMIZE_FLAG = True

algo_specs = []

def get_problem(PROBLEM_SIZE):
    STATE_LENGTH = PROBLEM_SIZE
    STATE_ALPHABET_SIZE = PROBLEM_SIZE
    fitness_func = mlrose.FourPeaks()
    return mlrose.DiscreteOpt(length=STATE_LENGTH, 
                              fitness_fn=fitness_func,
                              maximize=MAXIMIZE_FLAG,
                              max_val=STATE_ALPHABET_SIZE)
problem = get_problem(PROBLEM_SIZE)

RANDOM_STATE_INIT = False
INIT_STATE = None

PROBLEM_CODE = P_FOURPEAKS

In [0]:
# N Queens

PROBLEM_SIZE = 8
MAXIMIZE_FLAG = False

algo_specs = []

def get_problem(PROBLEM_SIZE):
  
    STATE_LENGTH = PROBLEM_SIZE
    STATE_ALPHABET_SIZE = PROBLEM_SIZE
    fitness_func = mlrose.Queens()
    return mlrose.DiscreteOpt(length=STATE_LENGTH, 
                              fitness_fn=fitness_func,
                              maximize=MAXIMIZE_FLAG,
                              max_val=STATE_ALPHABET_SIZE)
  
problem = get_problem(PROBLEM_SIZE)

RANDOM_STATE_INIT = False
INIT_STATE = None

PROBLEM_CODE = P_QUEENS

In [0]:
# Flip Flop
PROBLEM_SIZE = 100
MAXIMIZE_FLAG = True

algo_specs = []

def get_problem(PROBLEM_SIZE):
    fitness_func = mlrose.FlipFlop()
    return mlrose.DiscreteOpt(length=PROBLEM_SIZE, 
                              fitness_fn=fitness_func,
                              maximize=MAXIMIZE_FLAG,
                              max_val=2)
problem = get_problem(PROBLEM_SIZE)

RANDOM_STATE_INIT = False
INIT_STATE = None

PROBLEM_CODE = P_FLIPFLOP

### Algorithm & Hyperparameter Selection

In [0]:
# Randomized Hill Climbing Cell

######################
# Configurable stuff #
######################

RHC_NUM_TRIALS = 20

if PROBLEM_CODE == P_FOURPEAKS:
    rhc_hyperparam_list = [
        # Default hyperparams
       # {"max_attempts": 10000, "max_iters": 5000, "restarts": 0,},
       {"max_attempts": 1000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 100, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 10, "max_iters": 5000, "restarts": 0,},
    ]
elif PROBLEM_CODE == P_QUEENS:
    rhc_hyperparam_list = [
       # {"max_attempts": 10000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 1000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 100, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 10, "max_iters": 5000, "restarts": 0,},
       {"max_attempts": 1000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 1000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 100, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 10, "max_iters": 5000, "restarts": 5,},
       # {"max_attempts": 10000, "max_iters": 50000, "restarts": 0,},
       # {"max_attempts": 1000, "max_iters": 50000, "restarts": 5,},
       # {"max_attempts": 100, "max_iters": 50000, "restarts": 5,},
       # {"max_attempts": 10, "max_iters": 50000, "restarts": 5,},
    ]
elif PROBLEM_CODE == P_FLIPFLOP:
    rhc_hyperparam_list = [
       # {"max_attempts": 20, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 1000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 1000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 100, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 10, "max_iters": 5000, "restarts": 0,},
       {"max_attempts": 20, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 1000, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 100, "max_iters": 5000, "restarts": 0,},
       # {"max_attempts": 10, "max_iters": 5000, "restarts": 5,},
       # {"max_attempts": 1000, "max_iters": 50000, "restarts": 0,},
       # {"max_attempts": 1000, "max_iters": 50000, "restarts": 5,},
       # {"max_attempts": 100, "max_iters": 50000, "restarts": 5,},
       # {"max_attempts": 10, "max_iters": 50000, "restarts": 5,},
    ]
else:
    raise RuntimeError("Unrecognized Problem Code!")

rhc_specs = []
for rhc_hp in rhc_hyperparam_list:
    rhc_specs.append({"name": "RHC", 
                      "func": mlrose.random_hill_climb, 
                      "params": rhc_hp,
                      "trials": RHC_NUM_TRIALS})

In [0]:
# Simulating Annealing Cell

######################
# Configurable stuff #
######################

SA_NUM_TRIALS = 20

# The schedule object args are (init_temp, decay, min_temp)
# the decay should be high for geometric decay and low for exponential decay

if PROBLEM_CODE == P_FOURPEAKS:
    sa_hyperparam_list = [
        # After much selection, this is by far the best
        {"max_attempts": 1000, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .0005, .001)},
        # Default hyperparams
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .005, .001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .005, .01)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .05, .001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .3, .0001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .0005, .0001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .95, .001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .99, .001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .99, .1)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .8, .0001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .7, .001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .95, .01)},
    ]
elif PROBLEM_CODE == P_QUEENS:
    sa_hyperparam_list = [
        # {"max_attempts": 1000, "max_iters": 5000, "schedule": mlrose.GeomDecay(init_temp=1, decay=.0005, min_temp=.001)},
        # {"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .005, .001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .005, .01)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .05, .001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .3, .0001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .0005, .0001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .95, .001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .99, .001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .99, .1)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .8, .0001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .7, .001)},
        #{"max_attempts": 100, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .95, .01)},
    ]
elif PROBLEM_CODE == P_FLIPFLOP:
    sa_hyperparam_list = [
        # ACTUALLY THE BEST SO DONT LOSE THIS!!
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .3, .0001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .0005, .001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .0005, .001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .005, .001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .005, .01)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .05, .001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.ExpDecay(1, .0005, .0001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .95, .001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .99, .001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .99, .1)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .8, .0001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .7, .001)},
        # {"max_attempts": 20, "max_iters": 5000, "schedule": mlrose.GeomDecay(1, .95, .01)},
        {"max_attempts": 1000, "max_iters": 20, "schedule": mlrose.GeomDecay(init_temp=1, decay=.0005, min_temp=.0005)}
    ]
else:
    raise RuntimeError("Unrecognized code!")

sa_specs = []
for sa_hp in sa_hyperparam_list:
    sa_specs.append({"name": "SA", 
                     "func": mlrose.simulated_annealing, 
                     "params": sa_hp,
                     "trials": SA_NUM_TRIALS})

In [0]:
# Genetic Algorithm Cell

######################
# Configurable stuff #
######################

GA_NUM_TRIALS = 10

if PROBLEM_CODE == P_FOURPEAKS:
  ga_hyperparam_list = [
      # Default parameters work great, they pretty much work amazingly off the bat
      #{"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "mutation_prob": 0.1,},  
      
      # Pikachu I choose you!
      {"max_attempts": 1000, "max_iters": 5000, "pop_size": 400, "mutation_prob": 0.2,},
      #{"max_attempts": 1000, "max_iters": 5000, "pop_size": 50, "mutation_prob": 0.1,},
      #{"max_attempts": 10, "max_iters": 5000, "pop_size": 5000, "mutation_prob": 0.01,},
      #{"max_attempts": 10, "max_iters": 5000, "pop_size": 5000, "mutation_prob": 0.01,},
      #{"max_attempts": 10, "max_iters": 5000, "pop_size": 100, "mutation_prob": 0.3,},
      #{"max_attempts": 10, "max_iters": 5000, "pop_size": 100, "mutation_prob": 0.01,},
      #{"max_attempts": 100, "max_iters": 5000, "pop_size": 300, "mutation_prob": 0.001,},
  ]
elif PROBLEM_CODE == P_QUEENS:
  ga_hyperparam_list = [
      # Default parameters work great, they pretty much work amazingly off the bat
      {"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "mutation_prob": 0.1,},  
      # {"max_attempts": 1000, "max_iters": 5000, "pop_size": 400, "mutation_prob": 0.2,},
      # {"max_attempts": 1000, "max_iters": 5000, "pop_size": 50, "mutation_prob": 0.1,},
      # {"max_attempts": 10, "max_iters": 5000, "pop_size": 5000, "mutation_prob": 0.01,},
      # {"max_attempts": 10, "max_iters": 5000, "pop_size": 5000, "mutation_prob": 0.01,},
      # {"max_attempts": 10, "max_iters": 5000, "pop_size": 100, "mutation_prob": 0.3,},
      # {"max_attempts": 10, "max_iters": 5000, "pop_size": 100, "mutation_prob": 0.01,},
      # {"max_attempts": 100, "max_iters": 5000, "pop_size": 300, "mutation_prob": 0.001,},
  ]
elif PROBLEM_CODE == P_FLIPFLOP:
    ga_hyperparam_list = [
      # {"max_attempts": 20, "max_iters": 5000, "pop_size": 20, "mutation_prob": 0.1,},  
      # {"max_attempts": 20, "max_iters": 5000, "pop_size": 400, "mutation_prob": 0.2,},
      # {"max_attempts": 20, "max_iters": 5000, "pop_size": 50, "mutation_prob": 0.1,},
      {"max_attempts": 20, "max_iters": 5000, "pop_size": 5000, "mutation_prob": 0.01,},
      #{"max_attempts": 20, "max_iters": 5000, "pop_size": 5000, "mutation_prob": 0.01,},
      # { "max_attempts": 20, "max_iters": 5000, "pop_size": 100, "mutation_prob": 0.3,},
      # {"max_attempts": 20, "max_iters": 5000, "pop_size": 100, "mutation_prob": 0.01,},
      # {"max_attempts": 20, "max_iters": 5000, "pop_size": 300, "mutation_prob": 0.001,},
  ]
else:
    raise RuntimeError("Unrecgonized problem code: " + str(PROBLEM_CODE))

ga_specs = []
for ga_hp in ga_hyperparam_list:
    ga_specs.append({"name": "GA", 
                     "func": mlrose.genetic_alg, 
                     "params": ga_hp,
                     "trials": GA_NUM_TRIALS})

In [0]:
# Mimic Algorithm Cell

######################
# Configurable stuff #
######################

MIMIC_NUM_TRIALS = 10

if PROBLEM_CODE == P_FOURPEAKS:
  mimic_hyperparam_list = [
      
      # These hyperparameters provide pretty good seperation actually
      # {"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "keep_pct": 0.2,},
      # {"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "keep_pct": 0.46,},
      # {"max_attempts": 100, "max_iters": 5000, "pop_size": 400, "keep_pct": 0.30,},
      # {"max_attempts": 100, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.1,},
      # {"max_attempts": 50, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.7,},
      # {"max_attempts": 50, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.8,},
      
      # THE CHOSEN WAN
      {"max_attempts": 500, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.4,},
  ]
elif PROBLEM_CODE == P_QUEENS:
  mimic_hyperparam_list = [
      
      # These hyperparameters provide pretty good seperation actually
      # {"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "keep_pct": 0.2,},
      # {"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "keep_pct": 0.46,},
      # {"max_attempts": 100, "max_iters": 5000, "pop_size": 400, "keep_pct": 0.30,},
      # {"max_attempts": 100, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.1,},
      # {"max_attempts": 50, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.7,},
      # {"max_attempts": 50, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.8,},
      {"max_attempts": 500, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.4,},
  ]
elif PROBLEM_CODE == P_TSP:
  mimic_hyperparam_list = [
      # These hyperparameters provide pretty good seperation actually
      {"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "keep_pct": 0.2,},
      {"max_attempts": 1000, "max_iters": 5000, "pop_size": 200, "keep_pct": 0.46,},
      {"max_attempts": 100, "max_iters": 5000, "pop_size": 400, "keep_pct": 0.30,},
      {"max_attempts": 100, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.1,},
      {"max_attempts": 50, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.7,},
      {"max_attempts": 50, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.8,},
      {"max_attempts": 500, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.4,},
  ]
elif PROBLEM_CODE == P_FLIPFLOP:
    mimic_hyperparam_list = [
        # These hyperparameters provide pretty good seperation actually
        {"max_attempts": 20, "max_iters": 5000, "pop_size": 20, "keep_pct": 0.2,},
        # {"max_attempts": 20, "max_iters": 5000, "pop_size": 200, "keep_pct": 0.46,},
        # {"max_attempts": 20, "max_iters": 5000, "pop_size": 400, "keep_pct": 0.30,},
        # {"max_attempts": 20, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.1,},
        # {"max_attempts": 20, "max_iters": 5000, "pop_size": 1000, "keep_pct": 0.7,},
        # {"max_attempts": 20, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.8,},
        # {"max_attempts": 20, "max_iters": 5000, "pop_size": 2000, "keep_pct": 0.4,},
  ]
mimic_specs = []
for mimic_hp in mimic_hyperparam_list:
    mimic_specs.append({"name": "Mimic", 
                        "func": mlrose.mimic, 
                        "params": mimic_hp,
                        "trials": MIMIC_NUM_TRIALS})

### General Evaluation Cell

In [0]:
def eval_algos(problem, algo_specs, verbose=True):
    # Generic Evaluation Cell (Running that will evaluate all algorithms w/ all hyperparams)
    fitness_list = []
    history_list = []
    
    def print_v(*args):
        if verbose:
            print(*args)

    for index, algo_spec in enumerate(algo_specs):
        print_v("\n========================================================")
        print_v(algo_spec["name"] + " " + str(index))
        print_v("Hyperparams ", algo_spec["params"])

        start_time = time.time()

        num_trials = algo_spec["trials"]
        trial_fitnesses = [None] * num_trials
        trial_histories = [None] * num_trials

        for trial_index in range(algo_spec["trials"]):
            _, trial_fitnesses[trial_index], \
               trial_histories[trial_index] = algo_spec["func"](problem,
                                                                **algo_spec["params"])
        history_list.append(jagged_average(trial_histories))
        fitness_list.append(np.average(trial_fitnesses))

        end_time = time.time()
        print_v("--------------------------------------------------------")
        print_v("Total time spent training is " + str(end_time - start_time) + " seconds")
        print_v("Average peak fitness achieved was " + str(fitness_list[index]))
        
    return fitness_list, history_list

max_factor = 1 if MAXIMIZE_FLAG else -1

In [0]:
######################################################
# Evaluator Cell: Run all the configurations set     #
# above and graph performance vs iterations for each #
######################################################

algo_specs = rhc_specs + sa_specs + ga_specs + mimic_specs
fitness_list, history_list = eval_algos(problem, algo_specs)
algo_names = [algo_spec["name"] + " " + str(index) 
                  for index, algo_spec in enumerate(algo_specs)]

# Visualize the histories over time
# Graphing performance over time...
plt.title("Fitness vs Number Iterations")
plt.xlabel('Number of iterations')
plt.ylabel('Fitness')
# plt.ylim(ymin=0)

for index, history in enumerate(history_list):
    plt.plot(history[:500], label=algo_names[index])
    
pylab.legend(loc='lower right')
plt.show()

### Fitness vs Problem Size Experiment

In [0]:
# Test the effect of problem size on el problemo...
assert(len(rhc_specs) == len(sa_specs) == len(ga_specs) == len(mimic_specs) == 1)

###########################
# Configurable parameters #
###########################
if PROBLEM_CODE == P_FOURPEAKS or PROBLEM_CODE == P_QUEENS:
    MIN_PROBLEM_SIZE = 2
    MAX_PROBLEM_SIZE = 20
    SIZE_STEP = 2
    
if PROBLEM_CODE == P_FLIPFLOP:
    MIN_PROBLEM_SIZE = 5
    MAX_PROBLEM_SIZE = 100
    SIZE_STEP = 10

# Knock down the validation for the genetic algorithm and mimic, otherwise too long
ga_specs = copy.deepcopy(ga_specs)
mimic_specs = copy.deepcopy(mimic_specs)

ga_specs[0]["trials"] = 4
mimic_specs[0]["trials"] = 4
###########################
# End configuration stuff #
###########################
algo_specs = rhc_specs + sa_specs + ga_specs + mimic_specs

size_range = range(MIN_PROBLEM_SIZE, MAX_PROBLEM_SIZE, SIZE_STEP)
avg_fitnesses = [None] * len(size_range)

for index, size in enumerate(size_range):
    problem = get_problem(size)
    avg_fitnesses[index], _ = eval_algos(problem, algo_specs)
    
# Visualize algorithm performance over problem size
plt.title("Average Fitness vs Size of problem")
plt.xlabel('Problem Size')
plt.ylabel('Average Fitness')
# plt.ylim(ymin=0)
avg_fitnesses = np.transpose(np.array(avg_fitnesses))
plt.plot(size_range, avg_fitnesses[0], label="RHC")
plt.plot(size_range, avg_fitnesses[1], label="SA")
plt.plot(size_range, avg_fitnesses[2], label="GA")
plt.plot(size_range, avg_fitnesses[3], label="MIMIC")
    
# plt.ylim(ymin=0)
pylab.legend(loc='lower right')
plt.show()

### Effect of Varying Parameters on certain algorithms

In [0]:
if PROBLEM_CODE == P_QUEENS:
    changer_params = [
        {"baseline": rhc_specs, 
         "pkey": "max_attempts", "flavor": "Neighborhood Size",
         "min_val": 50, "max_val": 500, "pstep": 50, 
         "tform": (lambda x: int(x)),
         "trials": 200},

        {"baseline": sa_specs, 
         "pkey": "schedule", "flavor": "Temperature Decay",
         "min_val": 0.01, "max_val": .99, "pstep": .1, 
         "tform": (lambda x: mlrose.ExpDecay(.99, x, .001)),
         "trials": 200,},

        {"baseline": ga_specs, 
         "pkey": "mutation_prob", "flavor": "Mutation Probability",
         "min_val": 0.1, "max_val": .9, "pstep": .1, 
         "tform": (lambda x: x),
         "trials": 4,},

        #{"baseline": mimic_specs, 
        #"pkey": "keep_pct", "flavor": "Top Frac to keep",
        #"min_val": 0.05, "max_val": .2, "pstep": .025, 
        #"tform": (lambda x: x),
        #"trials": 10},
    ]
    
elif PROBLEM_CODE == P_FOURPEAKS:
    changer_params = [
        {"baseline": rhc_specs, 
         "pkey": "max_attempts", "flavor": "Neighborhood Size",
         "min_val": 5, "max_val": 500, "pstep": 5, 
         "tform": (lambda x: int(x)),
         "trials": 200},

        {"baseline": sa_specs, 
         "pkey": "schedule", "flavor": "Temperature Decay",
         "min_val": 0.01, "max_val": .99, "pstep": .1, 
         "tform": (lambda x: mlrose.ExpDecay(.99, x, .001)),
         "trials": 200,},

        {"baseline": ga_specs, 
         "pkey": "mutation_prob", "flavor": "Mutation Probability",
         "min_val": 0.1, "max_val": .9, "pstep": .1, 
         "tform": (lambda x: x),
         "trials": 4,},

        {"baseline": mimic_specs, 
        "pkey": "keep_pct", "flavor": "Top Frac to keep",
        "min_val": 0.05, "max_val": .2, "pstep": .025, 
        "tform": (lambda x: x),
        "trials": 4},
    ]
elif PROBLEM_CODE == P_FLIPFLOP:
    changer_params = [
        {"baseline": rhc_specs, 
        "pkey": "max_attempts", "flavor": "Neighborhood Size",
        "min_val": 50, "max_val": 500, "pstep": 50, 
        "tform": (lambda x: int(x)),
        "trials": 200},

        {"baseline": sa_specs, 
         "pkey": "schedule", "flavor": "Temperature Decay",
         "min_val": 0.01, "max_val": 100, "pstep": 1, 
         "tform": (lambda x: mlrose.ExpDecay(.99, x, .001)),
         "trials": 200,},

        {"baseline": ga_specs, 
         "pkey": "mutation_prob", "flavor": "Mutation Probability",
         "min_val": 0.1, "max_val": .9, "pstep": .1, 
         "tform": (lambda x: x),
         "trials": 4,},

        {"baseline": mimic_specs, 
        "pkey": "keep_pct", "flavor": "Top Frac to keep",
        "min_val": 0.02, "max_val": .2, "pstep": .03, 
        "tform": (lambda x: x),
        "trials": 4},
    ]

cp_problem = get_problem(PROBLEM_SIZE)
max_factor = 1 if MAXIMIZE_FLAG else -1

for index, cp in enumerate(changer_params):

    param_range = np.arange(cp["min_val"], cp["max_val"], cp["pstep"])
    avg_fitnesses = [None] * len(param_range)
    for learner_index, pval in enumerate(param_range):
        params = copy.deepcopy(cp["baseline"][0])
        params["params"][cp["pkey"]] = cp["tform"](pval)
        params["trials"] = cp["trials"]
        avg_fitnesses[learner_index], _ = eval_algos(problem, [params], verbose=False)
        
    # Graphing the Results!!!
    plt.figure(index + 1)
    #ax = plt.subplot(len(changer_params), 1, index + 1)
    plt.title(cp["baseline"][0]["name"] + ": Average Fitness vs " + cp["flavor"])
    plt.xlabel(cp["flavor"])
    plt.ylabel('Average Fitness')
    # plt.ylim(ymin=0)
    # avg_fitnesses = np.transpose(np.array(avg_fitnesses))
    plt.plot(param_range, np.multiply(max_factor, avg_fitnesses))
    plt.show()